In [94]:
# %matplotlib widget

from __future__ import annotations


import matplotlib.colors as mcolors
from test_utilities import test, TestDict

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 6: Trash Compactor ---</h2><p>After helping the Elves in the kitchen, you were taking a break and helping them re-enact a movie scene when you over-enthusiastically jumped into the garbage chute!</p>
<p>A brief fall later, you find yourself in a <span title="To your surprise, the smell isn't that bad.">garbage smasher</span>. Unfortunately, the door's been magnetically sealed.</p>
<p>As you try to find a way out, you are approached by a family of cephalopods! They're pretty sure they can get the door open, but it will take some time. While you wait, they're curious if you can help the youngest cephalopod with her <a href="/2021/day/18">math homework</a>.</p>
<p>Cephalopod math doesn't look that different from normal math. The math worksheet (your puzzle input) consists of a list of <em>problems</em>; each problem has a group of numbers that need to be either <em>added</em> (<code>+</code>) or <em>multiplied</em> (<code>*</code>) together.</p>
<p>However, the problems are arranged a little strangely; they seem to be presented next to each other in a very long horizontal list. For example:</p>
<pre><code>123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
</code></pre>
<p>Each problem's numbers are arranged vertically; at the bottom of the problem is the symbol for the operation that needs to be performed. Problems are separated by a full column of only spaces. The left/right alignment of numbers within each problem can be ignored.</p>
<p>So, this worksheet contains four problems:</p>
<ul>
<li><code>123</code> * <code>45</code> * <code>6</code> = <code><em>33210</em></code></li>
<li><code>328</code> + <code>64</code> + <code>98</code> = <code><em>490</em></code></li>
<li><code>51</code> * <code>387</code> * <code>215</code> = <code><em>4243455</em></code></li>
<li><code>64</code> + <code>23</code> + <code>314</code> = <code><em>401</em></code></li>
</ul>
<p>To check their work, cephalopod students are given the <em>grand total</em> of adding together all of the answers to the individual problems. In this worksheet, the grand total is <code>33210</code> + <code>490</code> + <code>4243455</code> + <code>401</code> = <code><em>4277556</em></code>.</p>
<p>Of course, the actual worksheet is <em>much</em> wider. You'll need to make sure to unroll it completely so that you can read the problems clearly.</p>
<p>Solve the problems on the math worksheet. <em>What is the grand total found by adding together all of the answers to the individual problems?</em></p>
</article>


In [95]:
from math import prod


tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
        123 328  51 64 
         45 64  387 23 
          6 98  215 314
        *   +   *   +  
        """,
        "expected": 4277556,
    },
]


@test(tests=tests)
def part_I(s: str) -> int:
    *worksheet, operations = [l.strip().split() for l in s.strip().splitlines()]

    worksheet = [[int(i) for i in row] for row in zip(*worksheet)]

    return sum(
        sum(line) if op == "+" else prod(line)
        for op, line in zip(operations, worksheet)
    )


Test Example passed, for part_I.
Success


In [96]:
with open("../input/day6.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 3785892992137


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3785892992137</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The big cephalopods come back to check on how things are going. When they see that your grand total doesn't match the one expected by the worksheet, they realize they forgot to explain how to read cephalopod math.</p>
<p>Cephalopod math is written <em>right-to-left in columns</em>. Each number is given in its own column, with the most significant digit at the top and the least significant digit at the bottom. (Problems are still separated with a column consisting only of spaces, and the symbol at the bottom of the problem is still the operator to use.)</p>
<p>Here's the example worksheet again:</p>
<pre><code>123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
</code></pre>
<p>Reading the problems right-to-left one column at a time, the problems are now quite different:</p>
<ul>
<li>The rightmost problem is <code>4</code> + <code>431</code> + <code>623</code> = <code><em>1058</em></code></li>
<li>The second problem from the right is <code>175</code> * <code>581</code> * <code>32</code> = <code><em>3253600</em></code></li>
<li>The third problem from the right is <code>8</code> + <code>248</code> + <code>369</code> = <code><em>625</em></code></li>
<li>Finally, the leftmost problem is <code>356</code> * <code>24</code> * <code>1</code> = <code><em>8544</em></code></li>
</ul>
<p>Now, the grand total is <code>1058</code> + <code>3253600</code> + <code>625</code> + <code>8544</code> = <code><em>3263827</em></code>.</p>
<p>Solve the problems on the math worksheet again. <em>What is the grand total found by adding together all of the answers to the individual problems?</em></p>
</article>


In [97]:
from math import prod
import re

from more_itertools import flatten, minmax


tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
        123 328  51 64 
         45 64  387 23 
          6 98  215 314
         *   +   *   +  
        """,
        "expected": 3263827,
    },
]


@test(tests=tests)
def part_II(s: str) -> int:
    *worksheet, operations = [l for l in s.splitlines() if l and not l.isspace()]

    operations = re.findall(r"[\+\*]+", operations)[::-1]

    columns = [
        [(m.start(), m.end()) for m in re.finditer(r"\d+", line)] for line in worksheet
    ]
    columns = [list(minmax(flatten(cols))) for cols in zip(*columns)]

    worksheet = [[line[fr:to] for fr, to in columns] for line in worksheet]
    worksheet = [list(map(list, c))[::-1] for c in zip(*worksheet)][::-1]
    worksheet = [[c[::-1] for c in zip(*r)] for r in worksheet]
    worksheet = [[int("".join(c)) for c in r] for r in worksheet]

    return sum(
        sum(line) if op == "+" else prod(line)
        for op, line in zip(operations, worksheet)
    )


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [98]:
print(f"Part II: {part_II(puzzle)}")

Part II: 7669802156452


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>7669802156452</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
